In [ ]:
%pip install "psycopg[binary,pool]" pytz requests


In [ ]:
import requests
import json

# download data
data = {}

# https://api.open-meteo.com/v1/forecast?latitude=47.712596&longitude=13.618029&hourly=temperature_2m,cloud_cover,rain,snowfall,snow_depth,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,relative_humidity_2m,dew_point_2m&timezone=Europe%2FBerlin&past_days=7

# returns JSON

url = "https://api.open-meteo.com/v1/forecast?latitude=47.712596&longitude=13.618029&hourly=temperature_2m,cloud_cover,rain,snowfall,snow_depth,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,relative_humidity_2m,dew_point_2m&timezone=Europe%2FBerlin&past_days=7"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()  # Automatically parses JSON into a Python dictionary
    print("Downloaded JSON successfully!")
    print(data)
else:
    print(f"Failed to download JSON. Status code: {response.status_code}")


In [ ]:
hourly_units = data['hourly_units']
hourly_units


Load CSV into a Python data structure for processing

In [ ]:
metrics = list(hourly_units.keys())
metrics

In [ ]:
import pandas as pd

In [ ]:
hourly = data["hourly"]
data_frame_setup = {}

for metric in metrics:
    data_frame_setup[metric] = hourly[metric]
    

In [ ]:
df = pd.DataFrame(data_frame_setup)

df['time'] = pd.to_datetime(df['time'])



In [ ]:
df

In [ ]:
import psycopg
import os
import csv

csv_file_path = "/home/martin/Downloads/open-meteo-47.70N13.51E474m(1).csv"


os.environ['PGSERVICEFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pg_service.conf"
os.environ['PGPASSFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pgpass"

with psycopg.connect(service='eeg-middleware') as conn:
    with conn.cursor() as cur:
        
        with open(csv_file_path, mode="r", newline="", encoding="utf-8") as csvfile:
            reader = csv.DictReader(csvfile)  # Reads header and returns each row as a dictionary

            for row in reader:
                print(row)  # You can also access individual columns like row["temperature_2m (°C)"]
            
                insert_sql = """
                    INSERT INTO weather_historicweatherdata (
                        timestamp,
                        temperature_2m,
                        cloud_cover,
                        direct_radiation,
                        rain,
                        snowfall,
                        cloud_cover_low,
                        cloud_cover_mid,
                        cloud_cover_high
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (timestamp) DO NOTHING;
                """

                data = (
                    row['time'],  # timestamp
                    row['temperature_2m (°C)'],
                    row['cloud_cover (%)'],
                    row['direct_radiation (W/m²)'],
                    row['rain (mm)'],
                    row['snowfall (cm)'],
                    row['cloud_cover_low (%)'],
                    row['cloud_cover_mid (%)'],
                    row['cloud_cover_high (%)'],
                )

                cur.execute(insert_sql, data)
                
            # for
        # open
            
        c = conn.commit()
        #print(c)
        print("Data inserted successfully.")
        
        